In [7]:
import json
import os
from detectron2.data import DatasetCatalog, MetadataCatalog
import random

## Limit Validation Images

In [8]:
SPLIT = "validation"
IN = "panoptic_full.json"
OUT = "panoptic.json"
MAX_PIXELS = 8000000
DISCARD_P = 0.0

with open(os.path.join(SPLIT, IN), "r") as handler:
    panoptic = json.load(handler)

    
image_res = {img["id"]: img["width"]*img["height"] for img in panoptic["images"]}
valid_images = {img["id"] for img in panoptic["images"] 
                if img["width"]*img["height"] <= MAX_PIXELS and random.random() > DISCARD_P}
panoptic["annotations"] = [annotation for annotation in panoptic["annotations"] 
                           if annotation["image_id"] in valid_images]
print(len(panoptic["annotations"]))
with open(os.path.join(SPLIT, OUT), "w") as handler:
    json.dump(panoptic, handler)

1232


## Add 0 category

In [3]:
SPLITS = ("training", "validation")
FILE = "panoptic.json"

for split in SPLITS:
    with open(os.path.join(split, FILE), "r") as handler:
        panoptic = json.load(handler)
    
    if panoptic["categories"][0]["id"] != 0:
        mock = {"id": 0, "name": "unlabeled", "supercategory": "unlabeled", "isthing": 0, "color": [0, 0, 0]}
        panoptic["categories"] = [mock] + panoptic["categories"]
    
    with open(os.path.join(split, FILE), "w") as handler:
        json.dump(panoptic, handler)